Copyright (c) 2023 Qualcomm Innovation Center, Inc. All rights reserved. <br>
SPDX-License-Identifier: BSD-3-Clause-Clear

**Takeaways** : Users will learn to profile inferences running on Cloud AI 100 at a system and device level <br><br>
**Before you start** : Some commands (folder locations etc) will need to be updated based on the platform and installation locations. 'Sudo' prefix might be required for some commands to run properly. <br> This notebook assumes that the user understands how to onboard a model, compile and run using qaic-exec and qaic-runner (AIC native tools) respectively. <br>
The terms 'model' and 'network' are used interchangeably in this notebook. 
The terms 'NSP' and 'AI compute core" are used interchangeably in this notebook. 

**Last Verified Qualcomm Cloud AI Platform SDK and Apps SDK Version:** Platform SDK 1.10.0.193 and Apps SDK 1.10.0.193 

# <span style='color:Blue'> Profiling Inferences </span>

##  Pre-requisite reading 
New users on Cloud AI platforms are expected to have a good understanding of the compilation (qaic-exec) and inference (qaic-runner, qaic-api-test) execution tools. 

## Introduction 
This notebook will cover inference profiling at system and device level using the 'bert-base-cased' model from HuggingFace. 

Here is the workflow that will be demonstrated in this notebook. 

1. **Install required packages and download the model**: Begin by installing the required packages and downloading the model.  
2. **Device health check**: Query the device health using qaic-util tool. 
3. **Device-level profiling** :
    - Compiler estimation of the resources used on device. 
    - Summary of the operators, its count and device cycles used. 
    - Detailed timeline view of inference execution on device
4. **System-level profiling**: Breakdown the inference time at system level - application, linux runtime, kernel and device.  



# <span style='color:Blue'> 1. Install required packages and download the model </span>

We will install the required Python packages 

In [1]:
import sys
sys.executable

'/opt/qti-aic/dev/python/qaic-env/bin/python'

In [2]:
!/opt/qti-aic/dev/python/qaic-env/bin/pip install -r requirements.txt

In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
!/opt/qti-aic/dev/python/qaic-env/bin/optimum-cli export onnx --model bert-base-cased --cache_dir model_files/cased --opset 11 --task question-answering bert_base_cased_onnx/

2023-07-31 20:40:05.868379: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-31 20:40:05.868405: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Framework not specified. Using pt to export to ONNX.
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using framework PyTorch: 1.11.0+cu102
Overriding 1 configuration item(s)
	- use_cache -> False
Post-processing the exported models...
Validating models in subprocesses...
2023-07-31 20:40:20.848526: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not 

In [4]:
## Clip the constant (used to represent -inf) out of FP16 range to FP16.   

import numpy as np
import onnx
from onnx import numpy_helper
        
def fix_onnx_fp16(
    gen_models_path: str,
    model_base_name: str,
) -> str:
    finfo = np.finfo(np.float16)
    fp16_max = finfo.max
    fp16_min = finfo.min

    model = onnx.load(f"{gen_models_path}/{model_base_name}.onnx")
    fp16_fix = False
    for tensor in onnx.external_data_helper._get_all_tensors(model):
        nptensor = numpy_helper.to_array(tensor, gen_models_path)
        if nptensor.dtype == np.float32 and (
            np.any(nptensor > fp16_max) or np.any(nptensor < fp16_min)
        ):
            print(f'tensor value : {nptensor} above {fp16_max} or below {fp16_min}')
            nptensor = np.clip(nptensor, fp16_min, fp16_max)
            new_tensor = numpy_helper.from_array(nptensor, tensor.name)
            tensor.CopyFrom(new_tensor)
            fp16_fix = True
            
    if fp16_fix:
        # Save FP16 model
        print("Found constants out of FP16 range, clipped to FP16 range")
        model_base_name += "_fix_outofrange_fp16"
        onnx.save(model, f=f"{gen_models_path}/{model_base_name}.onnx")
        print(f"Saving modified onnx file at {gen_models_path}/{model_base_name}.onnx")
    return model_base_name

fp16_model_name = fix_onnx_fp16(gen_models_path="bert_base_cased_onnx", model_base_name="model")

tensor value : -3.4028234663852886e+38 above 65504.0 or below -65504.0
Found constants out of FP16 range, clipped to FP16 range
Saving modified onnx file at bert_base_cased_onnx/model_fix_outofrange_fp16.onnx


# <span style='color:Blue'>2. Check Device Health </span>

'qaic-util' tool can be used to query the health of all the Cloud AI cards in the server. 

"Status: Ready" indicates that the health of the cards is good. 

"Status: Error" indicates that the cards are not in good health and a system administrator needs to be contacted to rectify the issue. 


In [5]:
!/opt/qti-aic/tools/qaic-util -q | grep "Status"

	Status:Ready
	Status:Ready
	Status:Ready
	Status:Ready
	Status:Ready
	Status:Ready
	Status:Ready
	Status:Ready


# <span style='color:Blue'>3. Device Level Profiling - Compiler Opstats </span>

The Compiler Operator Trace (OpStat) feature allows capturing of network operator trace information from the NSP AI cores combined with QAic Linux Runtime and QAic Kernel events from the host. The combined trace shows a timeline view of host processing along with operator execution and synchronization delays. To enable operator traces, the following parameters need to be added. The number of samples aic-profiling-num-samples and starting iteration aic-profiling-start-iter can be adjusted. The profiling output directory aic-profiling-out-dir must exist.

**qaic-exec**: -aic-profiling-type=stats
-aic-profiling-type=trace
-aic-profiling-type=latency
-aic-perf-metrics : prints an outline of the resource usage of the compiled network

**qaic-runner**: aic-profiling-type : latency
aic-profiling-out-dir : output directory for latency capture (needs to exist before this command is run)
aic-profiling-start-iter : Set this value high enough to start capturing samples after device warmup (~250mS)
aic-profiling-num-samples : # of samples to be captured. Can be set greater than # of inferences

**Sequence of Steps to be run**

1. Compile with the aic-profiling flags
2. Run with the aic-profiling flags

In this example JSON trace files are generated in the profiling output directory ./optraces. The trace files can be viewed with the Chrome/Chromium trace browser chrome://tracing.

In [6]:
!rm -rf compiled_fp16
!/opt/qti-aic/exec/qaic-exec -v -aic-hw  -convert-to-fp16 \
-mos=1 -ols=1 -aic-num-cores=2 \
-m=bert_base_cased_onnx/model_fix_outofrange_fp16.onnx \
-onnx-define-symbol=sequence_length,128 -onnx-define-symbol=batch_size,8 \
-aic-perf-metrics -multicast-weights -stats-batchsize=8 -aic-binary-dir=./compiled_fp16 \
-aic-hw-version=2.0 -compile-only \
-aic-profiling-type=stats -aic-profiling-type=trace -aic-profiling-type=latency 

Reading ONNX Model from bert_base_cased_onnx/model_fix_outofrange_fp16.onnx
Compile started ............... 
Compiling model with FP16 precision.
AIC WARNING: MAC estimate does not yet handle all operations
Original MAC estimate: 18446744060020916224
Original Graph operation size histogram:
output elts    op count (cumulative)
       1024                  13   2%
       2048                  15   2%
     786432                 333  64%
    1572864                 420  81%
    3145728                 516 100%
preD32 constants size: 342228040 B
AIC HMX MAC estimate:  18446744060044509184
AIC non-HMX MAC estimate:  0
d32 constants size: 215076032 B
final constants size: 215076032 B
Using VTCM alg: NOT fatness reduced first fit reset
MulticastTableSize: 6
VTCM0Size:   8323072 B
VTCM1Size:   8323072 B
VTCMMaxSize: 8323072 B
StaticSharedDDRSize: 	      886016 B
DynamicSharedDDRSize: 	           0 B
StaticConstantsSize: 	   215076864 B
DynamicConstantsSize: 	           0 B
Constants size: 215

**-aic-perf-metrics** dumps the compiler estimate of the usage of DDR vs VTCM (Vector Tightly Coupled Memory). The compiler tries to place the intermediate layer outputs and weights in VTCM. 'Constant only' refer to the weights of the network. The difference in 'DDR traffic in' and 'constant only' provides any spilover of intermediate outputs to DDR which impacts performance.


In [7]:
!mkdir bert_base_cased_stats
!/opt/qti-aic/exec/qaic-runner --test-data ./compiled_fp16  -d 0 -S 1 -a 7 \
--aic-profiling-type latency --aic-profiling-out-dir ./bert_base_cased_stats \
--aic-profiling-type stats --aic-profiling-type trace \
--aic-profiling-start-iter 100 --aic-profiling-num-samples 1 --time 20

#Note: The throughput will drop significantly with --aic-profiling-type ascii --aic-profiling-type trace flags

Writing file:./bert_base_cased_stats/aic-profiling-program-0-trace.json
Deleting previous file: ./bert_base_cased_stats/aic-profiling-program-0-trace.json
Writing file:./bert_base_cased_stats/aic-profiling-program-0-trace.json
Deleting previous file: ./bert_base_cased_stats/aic-profiling-program-0-trace.json
Writing file:./bert_base_cased_stats/aic-profiling-program-0-trace.json
Deleting previous file: ./bert_base_cased_stats/aic-profiling-program-0-trace.json
Writing file:./bert_base_cased_stats/aic-profiling-program-0-trace.json
Deleting previous file: ./bert_base_cased_stats/aic-profiling-program-0-trace.json
Writing file:./bert_base_cased_stats/aic-profiling-program-0-trace.json
Deleting previous file: ./bert_base_cased_stats/aic-profiling-program-0-trace.json
Writing file:./bert_base_cased_stats/aic-profiling-program-0-trace.json
Deleting previous file: ./bert_base_cased_stats/aic-profiling-program-0-trace.json
Writing file:./bert_base_cased_stats/aic-profiling-program-0-trace.jso

## Summary View 
A summary text file is created per instance (running on 1 or more NSP cores) showing distribution of the load on every NSP. There are 4 HVX(Vector Unit), 1 HMX(Matrix Unit) and 1 DMA engine per NSP. This stats shows % utilization of every engine with total number of cycles required for computation. It also has operator wise break down of cycles consumed.

In [8]:
!cat bert_base_cased_stats/aic-profiling-program-0-activation-0-inf-100.txt


core  ucycles  ucycles(%)     pcycles  pcycles(%)  count      out KB  ucycles/KB  pcycles/KB  stddev(pc/KB)  opdetails
c0     424682      95.50%         N/A         N/A    759         N/A         N/A         N/A            N/A  sync DMAIssue
c0     244552      55.00%    15612581      54.67%   1081   525124.00        0.47       29.73          20.98  aicconvolutiond32 HMX TCM (float16, i8, index32)
c0     186007      41.83%         N/A         N/A    906         N/A         N/A         N/A            N/A  sync HMX
c0     164066      36.90%         N/A         N/A    646         N/A         N/A         N/A            N/A  sync HVX
c0      70129      15.77%     4532059      15.87%     24    18432.00        3.80      245.88          11.79  aicsoftmax HVX TCM (float16, i8, index64)
c0      67009      15.07%       97200       0.34%    413    83189.12        0.81        1.17         104.81  aiccopytovtcm DMAIssue DDR 
c0      62830      14.13%     3950024      13.83%     72    36864.00       

## Detailed Timeline View
JSON files hold the detailed timeline view of the inference executed on device. 
Open chrome://tracing in Chrome/Chromium and click the Load button to open and view one of the JSON files. Use the toolbar to select, pan and zoom the timelines.

**Reading trace timeline** <br>
The trace timeline is divided into HOST_PROC and Process 0 .. N sections.

<div>
<img src="images/opstats_decoder.png" width="500"/>
</div>

**Viewing Operators** <br>
Zoom in to see individual operations. Here's an example showing convolution and softmax operations.

<div>
<img src="images/opstats_example.png" width="1000"/>
</div>

**Viewing Operator details** <br>
Click on an operator to see more details. Here are details of one of the softmax operations. 

<div>
<img src="images/operator_details.png" width="1000"/>
</div>

# <span style='color:Blue'>4. System Level Profiling </span>

System level profiling includes the breakdown of the inference time between Application, linux runtime , kernel mode driver and hardware processing.

qaic-runner tools can help with system level profiling. The following flags are required during inference. 

**qaic-runner** <br>
aic-profiling-type : latency <br>
aic-profiling-out-dir : output directory for latency capture (needs to exist before this command is run) <br>
aic-profiling-start-iter : Set this value high enough to start capturing samples after device warmup (~250mS) <br>
aic-profiling-num-samples : # of samples to be captured. Can be set greater than # of inferences <br><br>

<div>
<img src="images/Latency.png" width="1000"/>
</div>

In [9]:
!/opt/qti-aic/exec/qaic-runner --test-data ./compiled_fp16  -d 0 -S 1 -a 7 \
--aic-profiling-type latency --aic-profiling-out-dir ./bert_base_cased_stats \
--aic-profiling-start-iter 100 --aic-profiling-num-samples 99999 --time 20 

 ---- Stats ----
InferenceCnt 5787 TotalDuration 20018960us BatchSize 8 Inf/Sec 2312.608
Device Performance: 
Metric,          Avg,           Min,           Max,           Std            
ExecTimeUs,      22777.262,     22210.208,     25809.896,     374.926        
InputWaitTimeUs, 1403.480,      634.427,       2226.094,      233.809        
OutputWaitTimeUs,0.124,         0.104,         2.240,         0.090          

Deleting previous file: ./bert_base_cased_stats/aic-profiling-program-0-latency.txt
Writing file:./bert_base_cased_stats/aic-profiling-program-0-latency.txt
Deleting previous file: ./bert_base_cased_stats/aic-profiling-program-0-latency.txt
Writing file:./bert_base_cased_stats/aic-profiling-program-0-latency.txt
Deleting previous file: ./bert_base_cased_stats/aic-profiling-program-0-latency.txt
Writing file:./bert_base_cased_stats/aic-profiling-program-0-latency.txt
Deleting previous file: ./bert_base_cased_stats/aic-profiling-program-0-latency.txt
Writing file:./bert_ba

In [10]:
!python3 latency_stats_python3.py ./bert_base_cased_stats/aic-profiling-program-0-latency.txt 

All activations combined:
                          mean         min         50%         75%         90%         95%         99%         max
devInfPerSec        347.295312  309.959000  350.840500  353.190250  354.376700  354.804700  355.412240  355.981000
devExec              23.051370   22.473125   22.802370   23.164336   23.987068   24.345632   25.392607   25.809896
devInWaitTime         1.385587    0.721927    1.400572    1.520768    1.661437    1.742192    1.920286    2.226094
devOutWaitTime        0.000141    0.000104    0.000104    0.000104    0.000156    0.000365    0.000711    0.002083
hostRoundTrip        23.372801   22.652000   23.126000   23.505500   24.291600   24.672150   25.697940   26.089000
enqTime               0.006761    0.003787    0.006246    0.006934    0.007816    0.011465    0.019582    0.029266
preProcTime           0.007632    0.004348    0.006613    0.007150    0.011401    0.016421    0.018368    0.029383
submitTime            0.001556    0.000867    0.001410